# Initialize Kensho API Notebook

In [23]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Your API access token here
with open('/home/ubuntu/workspace/ex50/token/jeff') as f:
    secret_token = f.read().strip()

In [25]:
# %load ../examples/client.py
import os

import pandas as pd
import requests


def _api_result_to_frame(api_result):
    """Convert API result into dataframe"""
    ordered_columns = [c['key'] for c in api_result['metadata']]
    # Create the dataframe
    df = pd.DataFrame(api_result['data'])
    existence_filtered_columns = [c for c in ordered_columns if c in df.columns]
    reordered_frame = df[existence_filtered_columns]

    for column in api_result['metadata']:
        if column['key'] not in reordered_frame.columns:
            continue
        if column['unit'] == 'DateTime' or column['unit'] == 'Date':
            reordered_frame[column['key']] = pd.to_datetime(reordered_frame[column['key']])
    return reordered_frame


class _KenshoGraphClient(object):
    """Basic implementation of the Kensho Graph API client"""

    def __init__(self, host, api_key, converter_func, retries=1):
        """Initialize the client"""
        self._host = host
        self._api_key = api_key
        self._retries = retries
        self._converter_func = converter_func

    def list_entity_classes(self):
        """List all available entity classes"""
        return self._get_json_or_raise('list_entity_classes')

    def get_class_relationships(self, class_name):
        """Return all potential relationships of an entity of this class"""
        return self._get_json_or_raise(
            'get_class_relationships', class_name=class_name)

    def search_entities(self, class_name, search_string):
        """Search entities of the class_name that match search_string"""
        return self._get_json_or_raise(
            'search_entities', class_name=class_name, search_string=search_string)

    def list_entities_of_class(self, class_name):
        """List all entities of class"""
        return self._get_json_or_raise(
            'list_entities_of_class', class_name=class_name)

    def get_entity(self, entity_id):
        """Get entity by id"""
        return self._get_json_or_raise('get_entity', entity_id=entity_id)

    def get_related_entities(self, entity_id, relationship):
        """Get all entities related by one identified by entity_id by 'relationship'"""
        return self._get_json_or_raise(
            'get_related_entities', entity_id=entity_id, relationship=relationship)

    def get_timeline(self, timeline_id, start_date=None, end_date=None):
        """Get all events in a timeline. Optionally bound by start and end dates"""
        return self._get_json_or_raise(
            'get_timeline', timeline_id=timeline_id, start_date=start_date, end_date=end_date)

    def get_calendar(self, start_date, end_date):
        """Get all events happening in the interval [start_date, end_date)"""
        return self._get_json_or_raise('get_calendar', start_date=start_date, end_date=end_date)

    def get_ongoing_episodes(self, start_date, end_date):
        """Get all episodes ongoing in the interval [start_date, end_date)"""
        return self._get_json_or_raise(
            'get_ongoing_episodes', start_date=start_date, end_date=end_date)

    def translate_asset_id(self, id_type, asset_id):
        """Given an identification string for an asset return all known identifiers"""
        return self._get_json_or_raise('translate_asset_id', id_type=id_type, asset_id=asset_id)

    def _get_json_or_raise(self, function, **kwargs):
        """Get json from a given url. Uses get_with_retries underneath"""
        session = requests.Session()
        session.mount('https://', requests.adapters.HTTPAdapter(max_retries=self._retries))
        full_url = os.path.join(self._host, function)
        headers = {
            'Authorization': 'Token {}'.format(self._api_key),
            'Content-Type': 'application/json'
        }
        response = session.get(full_url, headers=headers, params=kwargs)
        response.raise_for_status()
        result_json = response.json()
        return self._converter_func(result_json)


def get_json_graph_client(host, api_key, retries=1):
    """Get a client that returns raw json"""
    return _KenshoGraphClient(host, api_key, lambda x: x, retries=retries)


def get_pandas_graph_client(host, api_key, retries=1):
    """Get a client that returns raw json"""
    return _KenshoGraphClient(
        host, api_key, _api_result_to_frame, retries=retries)


In [26]:
client = get_pandas_graph_client('http://www.kensho.com/external/v1', secret_token)
json_client = get_json_graph_client('http://www.kensho.com/external/v1', secret_token)

# Reference - basic classes

In [27]:
client.list_entity_classes()

,class_name,can_list_entities
0,Sector,True
1,Concept,True
2,TimelineType,True
3,Timeline,False
4,Region,True
5,Currency,True
6,BankMeeting,False
7,Equity,False
8,CentralBank,True


In [28]:
client.get_class_relationships('TimelineType')

,relationship_id,related_class
0,TimelineSubtype,TimelineType
1,TimelineTypeTimeline,Timeline


# Search all M&A events

### Get M&A timeline entity

In [30]:
df = client.list_entities_of_class('TimelineType')

In [31]:
df[df.entity_name.isin(['M&A'])]

,entity_name,entity_type,entity_id,entity_aliases,parent_name,parent_id
178,M&A,TimelineType,6205dfab-9d20-4933-b7ed-78b632bf1b64,[],Transactions,48fd28a4-5d49-45c8-a7e0-0654e03ed969


### Get all M&A Timelines

In [8]:
# Use 'TimelineTypeTimeline' relationship to get all events of certain timelinetype.
client.get_related_entities('6205dfab-9d20-4933-b7ed-78b632bf1b64', 'TimelineTypeTimeline')

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
0,Major Automotive Company Spinoff Milestones,Timeline,ffdc0dc3-4214-4631-98b6-53da408eff58,[],M&A,[]
1,U.S. Pharmaceutical Sector Broken Merger and A...,Timeline,6deb3dce-9561-4c08-825c-e84654b3057e,[],M&A,[]
2,DineEquity Asset Sale Announcements,Timeline,369b7158-ee21-4c62-a584-a2b29d7eec55,[],M&A,[]
3,Cameron International Corp Asset Sale Announce...,Timeline,89a9ba69-02fd-4a0e-8cfa-a854ac2defe2,[],M&A,[{'entity_id': '14b6d607-311a-4a7e-9442-bd2426...
4,The U.S. Department of the Treasury Sells Stak...,Timeline,66413a51-1a35-4160-b6e5-a2c4a77f4417,[],M&A,[{'entity_id': '229c72a7-d35a-4290-8853-f97248...
5,Major U.S. Private Equity Acquisitions,Timeline,8327b247-433d-40ac-8e75-eb98892a3fde,[],M&A,[]
6,U.S. Real Estate Sector Broken Merger and Acqu...,Timeline,cff0437a-459c-4e9b-8acf-d5157f0ac3b2,[],M&A,[]
7,Confirmed Apple Investments over $1M,Timeline,a0debd16-7178-4925-ba7e-2b4b82eded76,[],M&A,[{'entity_id': 'f9d09c10-c6de-4e8f-9624-996774...
8,U.S. Retail Sector Broken Merger and Acquisiti...,Timeline,f04b5101-8d92-45e6-bd08-2aa68667a566,[],M&A,[]
9,,Timeline,1ce25296-cbc9-43a4-b0df-11b55d699536,[],M&A,[{'entity_id': '190a9401-3a7f-4f3c-a49e-aa0487...


In [9]:
client.get_timeline('c2e5f461-de00-4876-b16e-b497a932816b')

,event_date,event_date_precision,event_name,provenance,primary_provenance,related_entities,event_description
0,1989-11-21,YYYY-MM-DD,Nissan invests $1 billion to expand its Mexico...,[http://articles.latimes.com/1989-11-22/busine...,[],[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
1,2001-03-20,YYYY-MM-DD,Nissan increases investment in Thailand,[],[https://newsroom.nissan-global.com/releases/0...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
2,2001-12-20,YYYY-MM-DD,Renault-Nissan invests $236 million in Brazil,[],[http://www.nissan-global.com/EN/NEWS/2001/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
3,2003-07-02,YYYY-MM-DD,Nissan invests 400 million euros in Nissan Mot...,[],[http://www.nissan-global.com/EN/NEWS/2003/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
4,2004-04-05,YYYY-MM-DD,Nissan increases investment in Thailand,[],[http://www.nissan-global.com/EN/NEWS/2004/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
5,2004-06-28,YYYY-MM-DD,Nissan invests $60 million in Egypt,[http://www.upi.com/Nissan-invests-60-million-...,[http://www.nissan-global.com/EN/NEWS/2004/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
6,2004-07-02,YYYY-MM-DD,Nissan invests about $250 million in Thai oper...,[],[http://www.nissan-global.com/EN/NEWS/2004/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
7,2004-07-16,YYYY-MM-DD,Nissan invests $47.3 million in plant in the U.S.,[ ],[http://www.nissan-global.com/EN/NEWS/2004/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
8,2004-11-29,YYYY-MM-DD,Nissan invests €140 million in new truck produ...,[http://www.just-auto.com/news/nissan-to-inves...,[],[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,
9,2005-02-01,YYYY-MM-DD,Nissan invests £223 million in U.K,[],[http://www.nissan-global.com/EN/NEWS/2005/_ST...,[{'entity_id': '7c6e24c8-fa38-45ec-abcd-7af634...,


# Search M&A events for specific asset

### Find entity_id of an asset

In [10]:
client.search_entities('Equity', 'D05')

,entity_name,entity_type,entity_id,entity_aliases,ticker_name,sector_name,sector_id
0,DBS Group Holdings Ltd (D05),Equity,41c9afcc-0906-4b77-a175-69a7b00d3021,[],D05,Diversified Banks,448a3161-f080-4684-8333-f2c74b5d77a8


### And it's M&A Timeline

In [11]:
dbs_events = client.get_related_entities('41c9afcc-0906-4b77-a175-69a7b00d3021', 'EquityTimelines')

In [13]:
#The timeline_type of M&A event is 'M&A'
dbs_events[dbs_events.timeline_type == 'M&A']

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
1,Mergers and Acquisitions announcements by DBS ...,Timeline,a869291b-6ead-4f56-84f8-bff4548e9594,[],M&A,[{'entity_id': '41c9afcc-0906-4b77-a175-69a7b0...


In [14]:
client.get_timeline('a869291b-6ead-4f56-84f8-bff4548e9594')

,event_date,event_date_precision,event_name,related_entities,primary_provenance,provenance,event_description
0,2001-04-09,YYYY-MM-DD,DBS Group Holdings Ltd announces plans to acqu...,[{'entity_id': '41c9afcc-0906-4b77-a175-69a7b0...,[],[],
1,2008-02-01,YYYY-MM-DD,DBS Group Holdings Ltd announces plans to acqu...,[{'entity_id': '41c9afcc-0906-4b77-a175-69a7b0...,[],[],


# Search M&A events in region

### Look up Japan M&A events entity

In [32]:
regions = client.list_entities_of_class('Region')

In [33]:
regions[regions.entity_name == 'Japan']

,entity_name,entity_type,entity_id,entity_aliases,region_type,parent_name,parent_id
223,Japan,Region,144276e1-01aa-457f-8c05-5f6350ad0434,"[JP, Japanese]",Country,Eastern Asia,2d4ecaf6-68e3-41bd-8cca-d5f1020d5992


In [34]:
client.get_class_relationships('Region')

,relationship_id,related_class
0,ContainedRegions,Region
1,Subregion,Region
2,RegionCurrency,Currency
3,RegionTimelines,Timeline


In [35]:
japan_timelines = client.get_related_entities('144276e1-01aa-457f-8c05-5f6350ad0434', 'RegionTimelines')

In [36]:
japan_timelines[japan_timelines.timeline_type == 'M&A']

,entity_name,entity_type,entity_id,entity_aliases,timeline_type,related_entities
30,Mergers and Acquisitions by Companies in Japan,Timeline,7e34e1fb-7305-47f3-862c-ca79b3169475,[],M&A,[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...


### List up all M&A events related to Japan

In [19]:
japan_mergers = client.get_timeline('7e34e1fb-7305-47f3-862c-ca79b3169475')

In [20]:
japan_mergers

,event_date,event_date_precision,event_name,acquired_company_region,primary_provenance,provenance,related_entities,event_description
0,1987-05-27,YYYY-MM-DD,Shimizu Corporation announces plans to acquire...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
1,1988-10-12,YYYY-MM-DD,"Hitachi Metals, Ltd. announces plans to acquir...",United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
2,1989-09-28,YYYY-MM-DD,Kyocera Corporation announces plans to acquire...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
3,1990-01-01,YYYY-MM-DD,Kikkoman Corporation announces plans to acquir...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
4,1990-11-26,YYYY-MM-DD,Panasonic Corporation announces plans to acqui...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
5,1991-10-30,YYYY-MM-DD,Shimadzu Corporation announces plans to acquir...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
6,1992-09-01,YYYY-MM-DD,Air Water Inc. announces plans to acquire 100....,Japan,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
7,1997-04-01,YYYY-MM-DD,"Ryohin Keikaku Co., Ltd. announces plans to ac...",United Kingdom of Great Britain and Northern I...,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
8,1997-06-30,YYYY-MM-DD,SoftBank Group Corp. announces plans to acquir...,United States of America,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,
9,1998-11-20,YYYY-MM-DD,"Mitsui O.S.K. Lines, Ltd. announces plans to a...",Japan,[],[],[{'entity_id': '144276e1-01aa-457f-8c05-5f6350...,


### Search for cross region M&A (Japan and another country)

In [50]:
japan_mergers[japan_mergers.acquired_company_region == 'Singapore']

,event_date,event_date_precision,event_name,acquired_company_region,primary_provenance,provenance,related_entities,event_description
186,2004-11-05,YYYY-MM-DD,"NTT DOCOMO, INC. announces plans to acquire 10...",Singapore,[],[],[],
274,2005-08-18,YYYY-MM-DD,"Mitsui O.S.K. Lines, Ltd. announces plans to a...",Singapore,[],[],[],
329,2006-01-05,YYYY-MM-DD,Toshiba Corporation announces plans to acquire...,Singapore,[],[],[],
446,2006-11-29,YYYY-MM-DD,Nidec Corporation announces plans to acquire 1...,Singapore,[],[],[],
486,2007-03-26,YYYY-MM-DD,Nidec Corporation announces plans to acquire 1...,Singapore,[],[],[],
492,2007-04-04,YYYY-MM-DD,Sumitomo Corporation announces plans to acquir...,Singapore,[],[],[],
591,2008-06-10,YYYY-MM-DD,"Toppan Printing Co., Ltd. announces plans to a...",Singapore,[],[],[],
616,2008-09-05,YYYY-MM-DD,"Kyushu Electric Power Company, Incorporated an...",Singapore,[],[],[],
813,2010-07-26,YYYY-MM-DD,"Kirin Holdings Company, Limited announces plan...",Singapore,[],[],[],
911,2011-07-13,YYYY-MM-DD,"Mitsui & Co., Ltd. announces plans to acquire ...",Singapore,[],[],[],
